In [53]:
DATA_IN = "../data/raw/rees46/_"
DATA_OUT = "../data/raw/rees46/"

import findspark
findspark.init()

In [54]:
#import pyspark 
from pyspark.sql import SparkSession 

# later on - probably add dtypes in load phase
from pyspark.sql.types import StringType, IntegerType,\
    DoubleType, StructType, StructField, DateType

In [55]:
#sc = pyspark.SparkContext()
ss = SparkSession.builder.getOrCreate()
# terminate # set parames # get or create
ss.sparkContext.setLogLevel("ERROR")
ss.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
#ss.sparkContext.getConf().getAll()

In [56]:
def get_sdf_info(sdf, name=""):
    rc = sdf.count(); cc = len(sdf.columns) # note - wrap this into a fuc
    print("The {} dataset has shape of ({},{}) and following dtypes\n{}".\
        format(name, rc, cc, sdf.dtypes))

In [57]:
import glob
for f in glob.glob(DATA_IN+"*.gz"):
    if "events" not in locals():
        events = ss.read.csv(f,header=True, inferSchema=True, nanValue="null")
    else:
        events.union(ss.read.csv(f,header=True, inferSchema=True, nanValue="null"))
#events = ss.read.csv(glob.glob(DATA_DIR+"*.gz")[2],header=True, inferSchema=True)
get_sdf_info(events)

The  dataset has shape of (67542878,9) and following dtypes
[('event_time', 'string'), ('event_type', 'string'), ('product_id', 'int'), ('category_id', 'bigint'), ('category_code', 'string'), ('brand', 'string'), ('price', 'double'), ('user_id', 'int'), ('user_session', 'string')]


In [58]:
# keep interactions for users with >=1 trans
from pyspark.sql.functions import col, countDistinct
#users_to_keep = events.filter(col("event_type")=="purchase").\
#    groupBy("user_id").agg(countDistinct("user_session").\
#        alias("n_transactions")).filter(col("n_transactions")>=1).select(col("user_id"))
#events = events.join(users_to_keep, ["user_id"], "inner")
#get_sdf_info(events)

In [59]:
# convert dates
from pyspark.sql.functions import to_timestamp
events = events.withColumn("event_time", to_timestamp(col("event_time")))
get_sdf_info(events)

The  dataset has shape of (67542878,9) and following dtypes
[('event_time', 'timestamp'), ('event_type', 'string'), ('product_id', 'int'), ('category_id', 'bigint'), ('category_code', 'string'), ('brand', 'string'), ('price', 'double'), ('user_id', 'int'), ('user_session', 'string')]


In [60]:
# carve-out product-cat info as it is duplicated
products = events.select(["product_id", "category_id", "brand"]).dropDuplicates()
categories = events.select(["category_id", "category_code"]).dropDuplicates()
events = events.select(["event_time", "event_type", "user_id", "product_id", "price","user_session"])
get_sdf_info(products, "products")
get_sdf_info(categories, "categories")
get_sdf_info(events, "events")

The products dataset has shape of (214999,3) and following dtypes
[('product_id', 'int'), ('category_id', 'bigint'), ('brand', 'string')]


The categories dataset has shape of (1162,2) and following dtypes
[('category_id', 'bigint'), ('category_code', 'string')]


The events dataset has shape of (67542878,6) and following dtypes
[('event_time', 'timestamp'), ('event_type', 'string'), ('user_id', 'int'), ('product_id', 'int'), ('price', 'double'), ('user_session', 'string')]


In [61]:
# try to save both of the datasets to gz
# NOTE: ADD HEADER YOU ANIMAL
import os
if not os.path.exists(DATA_OUT):
    os.makedirs(DATA_OUT)
events.write.csv(DATA_OUT+"events", compression="gzip", header=True)
products.write.csv(DATA_OUT+"products", compression="gzip", header=True)
categories.write.csv(DATA_OUT+"categories", compression="gzip", header=True)

In [ ]:
# the product table - both of them small enough to do this firstly in pandas and rewriting it to spark

# product id
# category id
# collapsed brand name

# category table

# category id
# category name

In [13]:
ss.read.csv(DATA_OUT+"products").count()

180907

In [46]:
ss.read.csv(DATA_OUT+"products").select("_c0").distinct().count()

176133

In [50]:
ss.read.csv(DATA_OUT+"products").select(["_c0","_c3"]).distinct().count()

180907

In [ ]:
# the products differ only in the brand name


In [41]:
ss.read.csv(DATA_OUT+"products").groupBy("_c0").count().orderBy("count").tail(20)

[Row(_c0='100035628', count=2),
 Row(_c0='1307579', count=2),
 Row(_c0='12719061', count=2),
 Row(_c0='29501900', count=2),
 Row(_c0='100055424', count=2),
 Row(_c0='100052322', count=2),
 Row(_c0='100062425', count=2),
 Row(_c0='100042748', count=2),
 Row(_c0='10501054', count=2),
 Row(_c0='100017901', count=2),
 Row(_c0='100061898', count=2),
 Row(_c0='100053117', count=2),
 Row(_c0='100040716', count=2),
 Row(_c0='100029997', count=2),
 Row(_c0='100027014', count=3),
 Row(_c0='100056240', count=3),
 Row(_c0='100027991', count=3),
 Row(_c0='100040178', count=3),
 Row(_c0='100046228', count=3),
 Row(_c0='100046224', count=3)]

In [51]:
ss.read.csv(DATA_OUT+"products").filter(col("_c0")==100046224).show()

+---------+-------------------+-------------+------+
|      _c0|                _c1|          _c2|   _c3|
+---------+-------------------+-------------+------+
|100046224|2053013558978937451|sport.bicycle|  null|
|100046224|2053013558978937451|sport.bicycle|  nike|
|100046224|2053013558978937451|sport.bicycle|jordan|
+---------+-------------------+-------------+------+



In [52]:
ss.read.csv(DATA_OUT+"products")._c2.factorize()

AnalysisException: Path does not exist: file:/mnt/Data/git_root/churn-modeling/data/intermediate/rees46/products

In [43]:
ss.read.csv(DATA_OUT+"products", header=True)

DataFrame[7203086: string, 2232732084546306225: string, furniture.living_room.chair: string, _c3: string]

In [16]:
from pyspark.sql.functions import col, countDistinct